Trabajo Practico Nº 1 Inteligencia Artificial

Alan Cornejo y Martin Paura

In [8]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.aima import PriorityQueue as AimaPriorityQueue

index = 0

In [ ]:
def priority_func(x: NodeHanoi):
    result = 0
    Cant_Disc = len(x.state.rods[0]) +  len(x.state.rods[1]) +  len(x.state.rods[2])
    for i in range(len(x.state.rods[2])):  #Si los discos ya estan en su posiciòn final los trato de dejar ahi
        if x.state.rods[2][i] ==  Cant_Disc - i:
            result += 1**Cant_Disc
    aux_rod = 1        
    for j in range(Cant_Disc-1,0,-1):      #Premio a que se coloquen los discos en una varilla (0 o 1) para poder sacar el disco mas grande a la varilla 2
        for i in range(len(x.state.rods[aux_rod])):
            if x.state.rods[aux_rod][i] ==  j - i:
                result += 1**j
        if(aux_rod == 1):
            aux_rod = 0
        else:
            aux_rod = 1
    return result

In [250]:
def breadth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks,cost=1)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks,cost=1)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos una cola FIFO con el nodo inicial
    frontier =  AimaPriorityQueue(order='max', f=priority_func)

    frontier.append(NodeHanoi(problem.initial))
    
    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while len(frontier) != 0:
        Struct_node = frontier.pop()
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(Struct_node[1].state)
        
        if problem.goal_test(Struct_node[1].state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": Struct_node[1].depth,
                "cost_total": Struct_node[1].state.accumulated_cost,
            }
            return Struct_node[1], metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in Struct_node[1].expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:               
                frontier.append(next_node)  # Corregido

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": Struct_node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics


Ejecutemos una búsqueda:

In [261]:
solution, metrics = breadth_first_search(number_disks=10)

Veamos algunas metricas:

In [262]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 26653
states_visited: 12064
nodes_in_frontier: 429
max_depth: 1026
cost_total: 1027.0


In [263]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 2
Move disk 4 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 1
Move disk 3 from 2 to 3
Move disk 1 from 1 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 5 from 1 to 2
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 2
Move disk 3 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 1
Move disk 4 from 3 to 2
Move disk 1 from 1 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 2
Move disk 6 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 1
Move disk 3 from 2 to 3
Move disk 1 from 1 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 4 from 2 to 1
Move disk 1 from 3 to 1
Move disk 2 from

Ahora ejecutemos el método que nos genera los archivos para el visualizador:

In [264]:
solution.generate_solution_for_simulator()

El método no retorna nada, pero en donde ejecutamos esta notebook se guardaron los dos archivos.

![files](./img/files.png)

Esos `JSONs` los movemos a la carpeta `simulator` y desde esa carpeta podemos ejecutar:


```bash

conda init powershell #en el powershell para poder utilizar los comandos de conda y poder cambiar de enviroment

(base) conda info --envs  #para ver los entornos disponibles

# conda environments:
#
                       C:\Users\mpaur\anaconda3
                       C:\Users\mpaur\anaconda3\envs\EntornoIA
base                 * C:\Users\mpaur\miniconda3


(base)conda activate C:\Users\mpaur\anaconda3\envs\EntornoIA

(EntornoIA) python ./simulation_hanoi.py

```

Para poder visualizar la solución encontrada.